### 네이버 뉴스 정보 가져오기

### 학습 내용 
 * 네이버 뉴스 여러개 단어 검색
 * 페이지별 언론사, 뉴스기사, URL

### 01. 웹 브라우저 띄우기

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

### 네이버 뉴스 검색

In [2]:
driver = webdriver.Chrome('chromedriver_90')

url = 'https://search.naver.com/search.naver?query=apple&where=news'
driver.get(url)

import time
time.sleep(3)  # 홈페이지 로딩 시간 3초

In [3]:

keyword_multi = ['2021년 여름 패션 트렌드', 
                 '2021여름유행패션', '2021년 여름 컬러']

pages = 11

In [4]:
### 뉴스 검색 : //*[@id="nx_query"]
### 검색 버튼 : //*[@id="lnb.searchForm"]/fieldset/button

In [5]:
from datetime import datetime

now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)
print(now.day - 1)
now_day = now.day
now_hour = now.hour
file_make_time

4


'2021_0705_18'

In [6]:
import re
from datetime import timedelta

In [7]:
def date_cal(now_time):
    now = datetime.now()

    # 시간이 포함되어 있고,
    chk_time = now_time.find("시간")
    chk_day = now_time.find("일")
    if chk_time>=0: 
        hour = int( re.findall("\d+", now_time)[0] )
        
        # 현재 시간 - 적힌 시간이 음수가 나오면 현재 날짜에 -1을 해 준다. 
        cal_hour = now.hour - hour
        # print(cal_hour)
        if cal_hour < 0:
            news_date = timedelta(days = 1)
            cal_now = now - news_date
            #print("현재 날짜 :",  cal_now )
        else:
            news_date = timedelta(days = 0)
            cal_now = now - news_date
            #print("현재 날짜 :",  cal_now )
            
        cal_date_time = "%04d-%02d-%02d" % (cal_now.year, cal_now.month, cal_now.day)
        print("뉴스 날짜 : ", cal_date_time)
    
    # 일이 포함되어 있고,
    elif chk_day>=0: 
        news_day = int( re.findall("\d+", now_time)[0] )
        # print(news_day)
        
        # 현재 날짜 -1일전을 계산한다.
        news_date = timedelta(days = news_day)
        cal_now = now - news_date
        
        cal_date_time = "%04d-%02d-%02d" % (cal_now.year, cal_now.month, cal_now.day)
        # print("뉴스 날짜 : ", cal_date_time)
    else:
        cal_date_time = now_time
    return cal_date_time    

In [8]:
# date_cal("3일 전")

In [9]:
def keyword_news_n(keyword1, now_time):
    now_page_num = 0
    all_company1 = []
    all_article1 = []
    all_link1 = []
    all_date1 = []
    all_content1 = []
    
    for i in range(1,pages,1):
        # 페이지 부분 정보 획득
        sel_num = driver.find_element_by_class_name("sc_page_inner")
        sel_all = sel_num.find_elements_by_class_name("btn")
        
        # 화면에 보이는 페이지 확인
        print("현재 페이지 : ", now_page_num+1, "키워드 : ", keyword1, "작성시간: ", now_time)
        len_all = len(sel_all)
        
        # 페이지 번호 확인 후, 해당되는 버튼 클릭
        for i in range(1, len_all+1 ):
            tmp_pre = '//*[@id="main_pack"]/div[2]/div/div/a[' + str(i) + ']'
            sel_num = driver.find_element_by_xpath(tmp_pre)
            sel_page_num = int(sel_num.text)
            if sel_page_num == (now_page_num+1):
                # 정보를 가져와서 클릭
                # 현재 페이지 입력
                sel_num.click()
                time.sleep(1)
                now_page_num += 1 
                print(now_page_num)
                break

        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        all_ul = soup.find("ul", class_="list_news").find_all("li", class_="bx")

        for one in all_ul:
            # 언론사
            com = one.find("a", class_="info press").text
            com_a = com.replace("언론사 선정", "")

            # print(one)
            tmp = one.find("a", class_="news_tit")

            article_n = tmp.text
            tmp_link = tmp.attrs['href']
            
            # 날짜
            date_one = one.find('div', class_='info_group').find("span", class_="info").text
            now_dates = date_cal(date_one)  # 날짜 계산
            
            #내용
            tmp1 = one.find('a', class_='api_txt_lines dsc_txt_wrap')
            cont = tmp1.text
            
            print(article_n, com_a, now_dates)
            print("url : ", tmp_link)

            all_company1.append(com_a)
            all_article1.append(article_n)
            all_link1.append(tmp_link)
            all_date1.append(now_dates)
            all_content1.append(cont)
            
        print(len(all_article))
    return all_company1, all_article1, all_link1, all_date1, all_content1

In [10]:
import pandas as pd
from datetime import datetime
import os

### 폴더 생성

In [11]:
now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)

try:
    if not os.path.exists(file_make_time):
        os.makedirs(file_make_time)
        print("folder creating ok")
except OSError:
    print("Error : Creating directory ", file_make_time)

folder creating ok


In [12]:
now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)

all_company = []
all_article = []
all_link = []
all_date = []
all_content = []

for one in keyword_multi:
    url = 'https://search.naver.com/search.naver?query=' + one + '&where=news'
    driver.get(url)
    time.sleep(3)
    
    sel_search = driver.find_element_by_xpath('//*[@id="nx_query"]') 
    sel_btn = driver.find_element_by_xpath('//*[@id="nx_search_form"]/fieldset/button') 
    
    sel_search.clear()
    sel_search.send_keys(one)
    sel_btn.click()
    
    all_company, all_article, all_link, all_date, all_content = keyword_news_n(one, 
                                                                  file_make_time)
    
    dict_dat = {"언론사": all_company, "기사명" : all_article, 
                "날짜":all_date, "URL":all_link, "내용":all_content}
    
    dat = pd.DataFrame(dict_dat)
    dat['키워드'] = one
    dat.columns = [ "언론사", "기사명", "날짜", "URL", "내용", '키워드']
    
    fname_csv = file_make_time + "/" + one + "_" + file_make_time +".csv"
    fname_xlsx = file_make_time + "/" + one + "_" + file_make_time +".xlsx"
    dat.to_csv(fname_csv, index=False)
    dat.to_excel(fname_xlsx, index=False)

현재 페이지 :  1 키워드 :  2021년 여름 패션 트렌드 작성시간:  2021_0705_18
1
이준석과 봉태규 패션, 누가 몹쓸 핏?… MZ세대 패션탐구 [23CM] 조선일보 2021.06.14.
url :  https://www.chosun.com/culture-life/watching/2021/06/14/PCBORSSZJVGRHDGFH5TIZU77UU/?utm_source=naver&utm_medium=referral&utm_campaign=naver-news
여름패션의 꽃 ‘프린트’ 2021 베스트 트렌드 스냅 2021.06.22.
url :  http://zine.istyle24.com/Fashion/FashionView.aspx?Idx=54193&Menu=4&_C_=23069
"패션·리빙·가전·식품 몽땅 최저가"…CJ온스타일 플러스, 6월 프로모션 뉴스1 2021.06.09.
url :  https://www.news1.kr/articles/?4333141
‘집콕 패션’ 패러다임 올 체인지 패션비즈 2021.06.09.
url :  http://www.fashionbiz.co.kr/RE/?cate=2&recom=2&idx=184782
2021년 '봄ㆍ여름' 패션 트렌드?…'클래식ㆍ시즌리스' 미디어SR 2021.01.15.
url :  http://www.mediasr.co.kr/news/articleView.html?idxno=65282
내년 상반기에도 ‘집콕’ 패션이 대세 동아일보 D1면 1단
url :  https://www.donga.com/news/article/all/20201217/104487062/1
패션 시장은 벌써 여름!…더운 날씨에 여름패션 매출 증가↑ 디지틀조선일보 2021.04.26.
url :  http://digitalchosun.dizzo.com/site/data/html_dir/2021/04/26/2021042680010.html
에슬레저룩부터 워크레저룩까지, 2021년 기억해야 할 패션 키워드 ‘편안함’ 아시아경제

8
홈쇼핑, 봄옷 입었다…단독 신상품으로 소비심리 노크 메트로신문 2021.02.18.
url :  http://www.metroseoul.co.kr/article/20210218500316
2020년 복종별 하반기 F/W전략 - 백화점 철수하고 '온택트' 향해 달려간다 한국섬유신문 2020.07.17.
url :  http://www.ktnews.com/news/articleView.html?idxno=115957
[주간#신상태그] 설화수·예화담·헤지스·손오공·롯데호텔 외 프라임경제 2020.06.05.
url :  http://www.newsprime.co.kr/news/article.html?no=506909
[김지수의 인터스텔라] “창조의 핵심은 정리, 큰 덩어리로 남기고 다 버려야... 조선일보 2019.06.15.
url :  http://news.chosun.com/site/data/html_dir/2019/06/14/2019061402328.html?utm_source=naver&utm_medium=original&utm_campaign=news
[오늘e유통] 신라면세점, 6월 '무착륙 관광비행 항공권 증정' 이벤트 실시 外 이뉴스투데이 2021.06.09.
url :  http://www.enewstoday.co.kr/news/articleView.html?idxno=1485964
MCM '빅 로고 러버 슬라이드' 출시 여성소비자신문 2021.06.23.
url :  http://www.wsobi.com/news/articleView.html?idxno=126393
지속 가능한 세상을 위하여 여성조선 2021.06.17.
url :  http://woman.chosun.com/client/news/viw.asp?cate=C02&mcate=M1001&nNewsNumb=20210677136
[SR유통] MCM, '빅 로고 러버 슬라이드' 출시 SR타임스 2021.06.23.
url :  http://www.srtimes.kr/news/article

5
24일부터 18일 동안 ‘대한민국 동행세일’ 한겨레 2021.06.16.
url :  http://www.hani.co.kr/arti/economy/consumer/999657.html
런웨이가 다 뭐야? 2020년을 강타한 월드 패션시장 핫이슈 8 패션엔 2020.12.31.
url :  http://www.fashionn.com/board/read_new.php?table=1004&number=35444
칼럼-패션의 환경 책임, 천연염색으로 답한다 패션저널 2021.01.06.
url :  http://okfashion.co.kr/detail.php?number=55756&thread=81r01r18
[주간#신상태그] 애경산업·스파오·GS샵·손오공·호텔신라 외 프라임경제 2021.06.25.
url :  http://www.newsprime.co.kr/news/article.html?no=547485
[Consumer Journal] 공항패션은 다음 기회에…대신 '리조트룩' 뽐내볼까 매일경제 C1면 TOP
url :  http://news.mk.co.kr/newsRead.php?no=608871&year=2021
내년 상반기에도 ‘집콕’ 패션이 대세 동아일보 D1면 1단
url :  https://www.donga.com/news/article/all/20201217/104487062/1
나를 표현하는 상징 ‘머리스타일’ 미주중앙일보 2021.06.25.
url :  http://www.koreadaily.com/news/read.asp?art_id=9478186
2021 S/S 파리패션위크, 88개 브랜드 참가 일정 공개 패션엔 2020.09.08.
url :  http://www.fashionn.com/board/read_new.php?table=1025&number=34395
2021 키워드 ‘브이노믹스’부터 ‘휴먼 터치’까지-2021년, 이 단어들을 주목... 매일경제 2020.11.05.
url :  http://news.mk.co.kr/new

2
인터파크, '2021년 가구 상반기 결산 통합 프로모션' 진행 G밸리뉴스 2021-06-29
url :  http://www.gvalley.co.kr/news/articleView.html?idxno=581483
2021 S/S 룩을 입고 즐기는 영화적 여름 GQ 2021-06-30
url :  https://www.gqkorea.co.kr/2021/06/30/2021-s-s-%eb%a3%a9%ec%9d%84-%ec%9e%85%ea%b3%a0-%ec%a6%90%ea%b8%b0%eb%8a%94-%ec%98%81%ed%99%94%ec%a0%81-%ec%97%ac%eb%a6%84/?utm_source=naver&utm_medium=partnership
"힘내라 대한민국!" 롯데백화점, 24일부터 여름 정기 바겐세일 이투데이 2021.06.22.
url :  https://www.etoday.co.kr/news/view/2037539
로에베, 여름 과일 모티브 가방과 데일리룩 폴라 이비자 컬렉션 전개 패션엔 2021-06-28
url :  http://www.fashionn.com/board/read_new.php?table=1006&number=37261
LG생활건강, 여름 메이크업…'VDL 스테인드 글라스 에디션' 출시 매일안전신문 2021-07-01
url :  http://www.idsn.co.kr/news/articleView.html?idxno=49775
무더위 여름철, 땀 흘리지 않고 집안일 해결하는 '쾌적 가전' 머니투데이 2021.06.24.
url :  http://news.mt.co.kr/mtview.php?no=2021062416491169524
인천시교육청부평도서관, 여름독서교실 운영 브레이크뉴스 2021-06-29
url :  http://www.breaknews.com/817088
월간 아웃도어 2021년 7월호 목차 아웃도어뉴스 2021.06.25.
url :  http://www.outdoornews.co.kr/news/article

9
보국전자, 5월 30일 브랜드데이 진행...여름가전 최대 53% 할인 G밸리뉴스 2021.05.27.
url :  http://www.gvalley.co.kr/news/articleView.html?idxno=580390
[칼럼] 코로나19 '컬러 백신', 폭주하는 팬데믹 제동 미디어리퍼블릭 2021.05.13.
url :  http://www.mrepublic.co.kr/news/articleView.html?idxno=82424
"무더운 여름 쿨하고 스타일리시하게"...로로피아나, '썸머 컬렉션' 출시 청년일보 2021.05.26.
url :  http://www.youthdaily.co.kr/news/article.html?no=72191
머릿결 손상 1도 없이 컬러 피스만으로 매일 분위기 여신 될 수 있는 '헤어밴... 인사이트 2021.05.13.
url :  https://www.insight.co.kr/news/338101
여름 성수기 앞둔 호텔업계, ‘얼리 서머’ 패키지로 고객몰이 데일리안 2021.05.19.
url :  https://www.dailian.co.kr/news/view/992684/?sc=Naver
한지민·전지현·차정원·신예원 주얼리 화보… 볼드함·컬러감으로 활력을 선... 디지틀조선일보 2021.05.10.
url :  http://digitalchosun.dizzo.com/site/data/html_dir/2021/05/09/2021050980023.html
"시원하고 트렌디하게"...'여름 사냥' 쿨링 제품 봇물 이코노믹리뷰 2021.05.23.
url :  https://www.econovill.com/news/articleView.html?idxno=533081
레이첼콕스, 여름 데일리룩 위한 핫섬머 스트랩 샌들 전개 패션엔 2021.05.24.
url :  http://www.fashionn.com/board/read_new.php?table=1006&number=36843
로라로라, 2021년 여름 '펫 컬렉션'